<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/langchain_Vicuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# モデルのダウンロード

In [ ]:
# モデルのダウンロード
!wget https://huggingface.co/TheBloke/vicuna-13B-v1.5-GGML/resolve/main/vicuna-13b-v1.5.ggmlv3.q5_K_M.bin
PATH_MODEL = "/content/vicuna-13b-v1.5.ggmlv3.q5_K_M.bin"

# llama-cpp-pythonでモデルを使用する

In [ ]:
%%capture
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [ ]:
# モデルのダウンロード
import llama_cpp
import ctypes

llm =llama_cpp.Llama(model_path=PATH_MODEL, n_gpu_layers=40)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
# ref:https://github.com/lm-sys/FastChat/blob/d578599c69d060e6d40943f1b5b72af98956092a/fastchat/conversation.py#L286-L299
TEMPLATE = """
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
全て日本語で返答を行います。

USER:TopYoutuberになるにはどうすればよいですか？
ASSISTANT:
"""

In [ ]:
# 推論の実行
output = llm(
    TEMPLATE,
    max_tokens=1024,
)

print(output["choices"][0]["text"])

Llama.generate: prefix-match hit


1. 自分の強みや特技を見つけ、それを活かしたコンテンツを作成する。
2. ユーザーとのコミュニケーションを大切にし、リアルな情報発信を行う。
3. 他のYouTuberと競合しない独自の個性を持つこと。
4. 定期的に動画を更新することで視聴者の注目度を高める。
5. SEO対策を行って、検索エンジンで上位表示されるようにする。


## langchainと組み合わせる

In [ ]:
%%capture
!pip install langchain

In [ ]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate

# Basic langchain

In [ ]:
TEMPLATE = """
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
全て日本語で返答を行います。

USER:{user_input}
ASSISTANT:
"""

In [ ]:
prompt = PromptTemplate.from_template(TEMPLATE)
formatted_prompt = prompt.format(user_input="こんにちは")

response = llm(prompt=formatted_prompt, max_tokens=2048)
print(response['choices'][0]["text"])

Llama.generate: prefix-match hit


こんにちは！どのようなお話がしたいか教えてください。


# Chain

In [ ]:
# GPU メモリ解放
import gc
del llm
gc.collect()

34

In [ ]:
# https://python.langchain.com/docs/integrations/llms/llamacpp#gpu
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
# from langchain.callbacks.manager import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=PATH_MODEL,
    n_gpu_layers=40,
    n_batch=128,
    max_tokens=4096,
    n_ctx=4096,
    # callback_manager=callback_manager, 何故か精度が落ちる
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
TEMPLATE = """
{system_input}

USER:{user_input}
ASSISTANT:
"""

prompt = PromptTemplate.from_template(TEMPLATE)

In [ ]:
# Chainの定義
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
print(
    chain.run(
        {"system_input": "あなたは親切な人工知能です。全て日本語で答えてください", "user_input": "日本の首都はどこでしょうか？"}
    )
)

日本の首都は東京です。


# Summarization

In [ ]:
# GPU メモリ解放
del chain
gc.collect()

10

In [ ]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

In [ ]:
TEMPLATE = """
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
全て日本語で返答を行います。

USER:次の文章の簡潔な要約をしてください:{text}
ASSISTANT:
"""

In [ ]:
prompt = PromptTemplate.from_template(TEMPLATE)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

In [ ]:
from langchain.document_loaders import WebBaseLoader

In [ ]:
loader = WebBaseLoader("https://note.com/nyanta123/n/ncd9dae432356")
docs = loader.load()

In [ ]:
print(stuff_chain.run(docs))

Llama.generate: prefix-match hit



この記事では、ReazonSpeechという日本語に特化した音AIを使って文字起こしを行い、度を介します。ReazonSpeechは、ワンセグ放送の画データから学させた19,000時間のデータを持つモデルで、日本語に特化したモデルとしては世界最高度をります。ReazonSpeechの特としては、パラメータが少なくても高度であることがげられます。ReazonSpeechを使った文字起こしの方法はで、Googleコラボ上で実行することができます。文字起こししたい動画ファイルをアップロードして前理を行うだけで、ReazonSpeechのモデルや必要なパッケージをインストールして実行することができます。この記事は、ReazonSpeechの利用方法と度についてしく解されており、日本語に特化した音AIとしての活用法が知りたい人におすすめです。


# gpt-3.5-turboで要約してみる

In [ ]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
%%capture
!pip install openai

In [ ]:
llm_chain = LLMChain(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k"), prompt=prompt)

docs = loader.load()
print(llm_chain.run(docs))

この文章は、日本語に特化した音声認識AI「ReazonSpeech」を使用して高精度の文字起こしを行った結果について紹介しています。ReazonSpeechは、株式会社レアゾン・ホールディングスが開発した音声認識に関するモデルやデータの総称であり、音声をテキストに変換するAIです。このモデルは、ワンセグ放送の録画データから学習されたもので、日本語に特化しており、世界最高精度の音声認識モデルとされています。また、ReazonSpeechは比較的低スペックな計算機でも動作することができます。具体的な使い方としては、Googleコラボ上で簡単に文字起こしを行うことができます。使用方法や感想、今後の課題についても記事で紹介されています。
